# Data Translation

The following code is employed for the purpose of automatically translating our German text data into English, utilising natural language processing. We are employing three different models for this task. 

### Housekeeping

Here, we import the data and packages that we need for the translation.

In [ ]:
# import packages
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from deep_translator import GoogleTranslator

In [ ]:
# import data 
data = pd.read_csv("../data/raw/test_data.csv") 

### MarianMT

Translation with the model MarianMT:

In [ ]:
# define the model that is used for translation
marian = "Helsinki-NLP/opus-mt-de-en"
model = MarianMTModel.from_pretrained(marian)
tokenizer = MarianTokenizer.from_pretrained(marian)

In [ ]:
# define function to tranlate text data
def translate_marian(text):
    # tokenize the text input 
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # generate translation 
    translated_tokens = model.generate(**inputs)
    # decode the translated tokens into string
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

In [ ]:
# apply the translation function to the "text" column
data['text_eng'] = data['text'].apply(translate_marian)

# write data into a file 
data.to_csv("../data/translated/marian.csv", index=False)

### MBart

Translation with the model MBart: 

In [ ]:
# define the model that is used for translation
mbart = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(mbart)
tokenizer = MBart50TokenizerFast.from_pretrained(mbart)

In [ ]:
# define function to tranlate text data
def translate_mbart(text):
    tokenizer.src_lang = "de_DE"  
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

In [ ]:
# apply the translation function 
data['text_eng'] = data['text'].apply(translate_mbart)

# write data into a file
data.to_csv("../data/translated/mbart.csv", index=False)

### Google Translator

Translation with Google Translator:

In [ ]:
# define function to translate text data
def translate_google(text):
    translated = GoogleTranslator(source='de', target='en').translate(text)
    return translated

In [ ]:
# apply the translation function 
data['text_eng'] = data['text'].apply(translate_google)

# write data into a file
data.to_csv("../data/translated/google.csv", index=False)